In [ ]:
from mylibraries import *
X_train = pd.read_csv('X_train_scaled.csv')
X_test = pd.read_csv('X_test_scaled.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

# Získanie názvov stĺpcov z pôvodného X_train
column_names = X_train.columns


In [ ]:

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

# Definovanie modelu DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier

# Inicializácia modelu XGBoost Classifier
xgb = XGBClassifier(gamma=0.1, learning_rate=0.1, max_depth=7, min_child_weight=3, n_estimators=200)

# Trénovanie modelu
xgb.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred_xgb = xgb.predict(X_test)

# Vyhodnotenie modelu
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"Presnosť modelu XGBoost Classifier: {accuracy_xgb:.4f}")

# Klasifikačná správa
print("Klasifikačná správa modelu XGBoost Classifier:")
print(classification_report(y_test, y_pred_xgb))

# Matica zámen
print("Matica zámen modelu XGBoost Classifier:")
print(confusion_matrix(y_test, y_pred_xgb))

# ROC krivka
y_prob_xgb = xgb.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_prob_xgb)
plt.plot(fpr, tpr, label='ROC krivka (AUC = {:.2f})'.format(roc_auc_score(y_test, y_prob_xgb)))
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('Falošne pozitívna miera')
plt.ylabel('Pravdivo pozitívna miera')
plt.title('ROC Krivka pre XGBoost Classifier')
plt.legend()
plt.show()

# Precision-Recall krivka
precision, recall, _ = precision_recall_curve(y_test, y_prob_xgb)
average_precision = average_precision_score(y_test, y_prob_xgb)
plt.plot(recall, precision, label='Precision-Recall krivka (priemer = {:.2f})'.format(average_precision))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Krivka pre XGBoost Classifier')
plt.legend()
plt.show()

# Learning Curve
train_sizes, train_scores, test_scores = learning_curve(xgb, X_train, y_train, cv=5, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 10))
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.figure()
plt.plot(train_sizes, train_scores_mean, 'o-', color='r', label='Trénovacia presnosť')
plt.plot(train_sizes, test_scores_mean, 'o-', color='g', label='Validačná presnosť')

plt.xlabel('Počet trénovacích príkladov')
plt.ylabel('Presnosť')
plt.title('Learning Curve pre XGBoost Classifier')
plt.legend(loc="best")
plt.show()

# Dôležitosť atribútov
feature_importances = xgb.feature_importances_
sorted_idx = np.argsort(feature_importances)[::-1]
top_n = 10
top_features = column_names[sorted_idx][:top_n]

plt.barh(range(top_n), feature_importances[sorted_idx][:top_n])
plt.yticks(range(top_n), column_names[sorted_idx][:top_n])
plt.xlabel('Dôležitosť atribútov')
plt.title('Top {} najdôležitejších atribútov pre XGBoost Classifier'.format(top_n))
plt.show()

In [ ]:

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# Získanie dôležitosti atribútov
importances = xgb.feature_importances_

# Zoradenie atribútov podľa ich dôležitosti
sorted_idx = np.argsort(importances)[::-1]

# Výber najdôležitejších atribútov
top_n = 10
top_features = column_names[sorted_idx][:top_n]

# Rozdelenie dát na trénovaciu a testovaciu množinu len s najdôležitejšími atribútmi
X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

# Inicializácia modelu XGBoost Classifier
xgb_top = XGBClassifier(gamma=0.1, learning_rate=0.1, max_depth=7, min_child_weight=3, n_estimators=200)

# Trénovanie modelu na najdôležitejších atribútoch
xgb_top.fit(X_train_top, y_train)

# Predikcia na testovacej množine
y_pred_xgb_top = xgb_top.predict(X_test_top)

# Vyhodnotenie modelu
accuracy_xgb_top = accuracy_score(y_test, y_pred_xgb_top)
print(f"Presnosť modelu XGBoost Classifier na najdôležitejších atribútoch: {accuracy_xgb_top:.4f}")

# Klasifikačná správa
print("Klasifikačná správa modelu XGBoost Classifier na najdôležitejších atribútoch:")
print(classification_report(y_test, y_pred_xgb_top))

# Matica zámen
print("Matica zámen modelu XGBoost Classifier na najdôležitejších atribútoch:")
print(confusion_matrix(y_test, y_pred_xgb_top))

In [ ]:
# %%
from sklearn.ensemble import BaggingClassifier

# Inicializácia modelu Bagging Classifier s XGBoost ako odhadcom
bagging_xgb = BaggingClassifier(estimator=XGBClassifier(gamma=0.1, learning_rate=0.1, max_depth=7, min_child_weight=3, n_estimators=200), n_estimators=10, random_state=42)

# Trénovanie modelu Bagging s XGBoost na trénovacích dátach
bagging_xgb.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred_bagging_xgb = bagging_xgb.predict(X_test)

# Vyhodnotenie modelu
accuracy_bagging_xgb = accuracy_score(y_test, y_pred_bagging_xgb)
print(f"Presnosť modelu Bagging s XGBoost: {accuracy_bagging_xgb:.4f}")

# Klasifikačná správa
print("Klasifikačná správa modelu Bagging s XGBoost:")
print(classification_report(y_test, y_pred_bagging_xgb))

# Matica zámen
print("Matica zámen modelu Bagging s XGBoost:")
print(confusion_matrix(y_test, y_pred_bagging_xgb))